In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

# read 데이터
df = pd.read_csv('AFSNT_DLY.CSV', encoding="cp949")
df.head()


,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP,ODP,FLO,FLT,AOD,STT,DLY,DLY_RATE
0,2019,9,16,월,ARP1,ARP3,L,L1702,A,9:05,NaN,NaN
1,2019,9,16,월,ARP3,ARP1,L,L1702,D,7:55,NaN,NaN
2,2019,9,16,월,`,ARP3,L,L1720,A,14:40,NaN,NaN
3,2019,9,16,월,ARP3,ARP1,L,L1720,D,13:30,NaN,NaN
4,2019,9,16,월,ARP4,ARP3,L,L1808,A,20:10,NaN,NaN


In [3]:
# 필요없는 column 제거
df.drop(columns=['FLT', 'DLY', 'DLY_RATE'], axis=1, inplace=True)

In [4]:
df.head()

,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP,ODP,FLO,AOD,STT
0,2019,9,16,월,ARP1,ARP3,L,A,9:05
1,2019,9,16,월,ARP3,ARP1,L,D,7:55
2,2019,9,16,월,`,ARP3,L,A,14:40
3,2019,9,16,월,ARP3,ARP1,L,D,13:30
4,2019,9,16,월,ARP4,ARP3,L,A,20:10


In [5]:
# 파생변수
df['ARPODP'] = df['ARP'] + '_' + df['ODP']

In [6]:
df.head()

,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP,ODP,FLO,AOD,STT,ARPODP
0,2019,9,16,월,ARP1,ARP3,L,A,9:05,ARP1_ARP3
1,2019,9,16,월,ARP3,ARP1,L,D,7:55,ARP3_ARP1
2,2019,9,16,월,`,ARP3,L,A,14:40,`_ARP3
3,2019,9,16,월,ARP3,ARP1,L,D,13:30,ARP3_ARP1
4,2019,9,16,월,ARP4,ARP3,L,A,20:10,ARP4_ARP3


In [7]:
# 분 삭제
df['STT'] = pd.to_datetime(df['STT'],format= '%H:%M').dt.hour

In [8]:
import urllib
import json
import pandas as pd
import dateutil.parser
#날씨데이터 json 읽어오는 함수
#나중에 9월 1일을 9월 16일로 바꾼 후 실행..
def readJSON(area):
    
    request = urllib.request.urlopen('https://api.aerisapi.com/forecasts/'+area+',korea?from=09/01/2019&format=json&filter=1hr&limit=999&client_id=gHOhinKWCL1fwDUpI1Ec7&client_secret=TRamBTyXpORXcmDKVqc2S4i4mnCjVxxHMt6cllui')
    response = request.read()
    data= json.loads(response)


    if data['success']:
        return data

    else:
        print("An error occurred: %s" % (data['error']['description']))
        return ""
        request.close()

In [9]:
#날씨 csv 저장
def storeWeather():
    df=pd.DataFrame(columns=['SDT_YY', 'SDT_MM', 'SDT_DD','hour','ARP','temp', 'hum', 'dew', 'windSpeed'])

    for i in range(15):
        area=["seoul","busan","jeju","daegu","ulsan","cheongju","muan","gwangju","yeosu"
              ,"yangyang","pohang","sacheon","gunsan","wonju","incheon"]
        d=readJSON(area[i])
        response=d['response'][0]
        data=response['periods']
    
    
        for j in range(len(data)):
            temp=data[j]
            date=dateutil.parser.parse(temp['dateTimeISO'])
            # 바꿔야해요!! test 용
            temp2=pd.Series([date.year,date.month,date.day+15,date.hour,"ARP"+str(i+1),temp['tempC'],
                         temp['humidity'],temp['dewpointC'],temp['windGustKTS']],
                        index=['SDT_YY', 'SDT_MM', 'SDT_DD','hour','ARP','temp', 'hum', 'dew', 'windSpeed'])
        
            df=df.append(temp2,ignore_index=True)
    
    #fog 모델하고의 단위 맞추는 작업
    df['windSpeed']=df['windSpeed'].astype("float")*1852/3600
    
    df.to_csv("newWeather.csv")

In [10]:
# csv 파일 저장하고 싶을때만 실행..
storeWeather() 

# 날씨데이터 합치기---------------------
dly= pd.read_csv('AFSNT_DLY.csv', encoding="cp949")
df=pd.read_csv('newWeather.csv', encoding="cp949")

dly['hour'] = pd.to_datetime(dly['STT'],format= '%H:%M').dt.hour
merged= pd.merge(df, dly, on=['SDT_YY', 'SDT_MM', 'SDT_DD', 'ARP','hour'])
merged

# 일, 월, 화 -> 영어로 바꾸기


#---------------------------------------

,Unnamed: 0,SDT_YY,SDT_MM,SDT_DD,hour,ARP,temp,hum,dew,windSpeed,SDT_DY,ODP,FLO,FLT,AOD,STT,DLY,DLY_RATE
0,6,2019,9,16,6,ARP1,22,64,15,1.028889,월,ARP3,L,L1701,D,6:40,NaN,NaN
1,6,2019,9,16,6,ARP1,22,64,15,1.028889,월,ARP12,J,J1634,D,6:50,NaN,NaN
2,6,2019,9,16,6,ARP1,22,64,15,1.028889,월,ARP3,F,F1201,D,6:10,NaN,NaN
3,6,2019,9,16,6,ARP1,22,64,15,1.028889,월,ARP3,F,F1203,D,6:15,NaN,NaN
4,6,2019,9,16,6,ARP1,22,64,15,1.028889,월,ARP3,J,J1204,D,6:20,NaN,NaN
5,6,2019,9,16,6,ARP1,22,64,15,1.028889,월,ARP3,A,A1901,D,6:05,NaN,NaN
6,6,2019,9,16,6,ARP1,22,64,15,1.028889,월,ARP3,H,H1151,D,6:15,NaN,NaN
7,6,2019,9,16,6,ARP1,22,64,15,1.028889,월,ARP3,F,F1205,D,6:50,NaN,NaN
8,6,2019,9,16,6,ARP1,22,64,15,1.028889,월,ARP3,A,A1903,D,6:25,NaN,NaN
9,6,2019,9,16,6,ARP1,22,64,15,1.028889,월,ARP3,H,H1101,D,6:25,NaN,NaN


In [12]:
import warnings
warnings.filterwarnings(action="ignore")

for i in range(6):
    update=[]
    for j in range(24):
        newTemp=0.0
        newHum=0.0
        newDew=0.0
        newWindSpeed=0.0
        for k in range(5):
            newTemp += merged[(merged['SDT_DD']==25+i-k+1) & (merged['hour']==j)]['temp']
            newHum += merged[(merged['SDT_DD']==25+i-k+1) & (merged['hour']==j)]['hum']
            newDew += merged[(merged['SDT_DD']==25+i-k+1) & (merged['hour']==j)]['dew']
            newWindSpeed += merged[(merged['SDT_DD']==25+i-k+1) & (merged['hour']==j)]['windSpeed']
#            print(merged[(merged['SDT_DD']==25+i-k) & (merged['hour']==j)]['temp'])
        merged[(merged['SDT_DD']==25+i) & (merged['hour']==j)]['temp']= newTemp / 5
        merged[(merged['SDT_DD']==25+i) & (merged['hour']==j)]['hum']= newHum / 5
        merged[(merged['SDT_DD']==25+i) & (merged['hour']==j)]['dew']= newDew / 5
        merged[(merged['SDT_DD']==25+i) & (merged['hour']==j)]['windSpeed']= newWindSpeed / 5
    
print(merged)


       Unnamed: 0  SDT_YY  SDT_MM  SDT_DD  hour    ARP  temp  hum  dew  \
0               6    2019       9      16     6   ARP1    22   64   15   
1               6    2019       9      16     6   ARP1    22   64   15   
2               6    2019       9      16     6   ARP1    22   64   15   
3               6    2019       9      16     6   ARP1    22   64   15   
4               6    2019       9      16     6   ARP1    22   64   15   
...           ...     ...     ...     ...   ...    ...   ...  ...  ...   
15211        5186    2019       9      30     6  ARP15    19   70   13   
15212        5188    2019       9      30     8  ARP15    20   66   14   
15213        5188    2019       9      30     8  ARP15    20   66   14   
15214        5188    2019       9      30     8  ARP15    20   66   14   
15215        5188    2019       9      30     8  ARP15    20   66   14   

       windSpeed SDT_DY    ODP FLO    FLT AOD   STT  DLY  DLY_RATE  
0       1.028889      월   ARP3   L  L1701 

In [15]:
# YY랑 DD삭제
df.drop(columns=['SDT_YY', 'SDT_DD'], axis=1, inplace=True)

In [17]:
def one_hot_dummies(df, *args):
    for col in args:
        one_hot_col = pd.get_dummies(df[col])
        new_col = [col + '_' + str(s) for s in one_hot_col.columns]
        one_hot_col.columns = new_col
        df = df.drop([col], axis = 1)
        df = df.join(one_hot_col)

    return df

df = one_hot_dummies(df, 'SDT_MM', 'SDT_DY', 'ARP', 'ODP', 'FLO', 'ARPODP', 'STT')

In [22]:
df.columns

# 중요도 낮은 변수들 제거 

Index(['AOD', 'SDT_MM_9', 'SDT_DY_금', 'SDT_DY_목', 'SDT_DY_수', 'SDT_DY_월',
       'SDT_DY_일', 'SDT_DY_토', 'SDT_DY_화', 'ARP_ARP1',
       ...
       'STT_13', 'STT_14', 'STT_15', 'STT_16', 'STT_17', 'STT_18', 'STT_19',
       'STT_20', 'STT_21', 'STT_22'],
      dtype='object', length=110)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def fogModel(df):
    df.fillna(0, inplace = True)
    
    scaler = MinMaxScaler()
    df[['hum', 'dew','temp','windSpeed']] = scaler.fit_transform(df[['hum', 'dew','temp','windSpeed']])
    
    # 저장된 모델 불러오기 --------------
    
    # 모델 실행 
    
    #return 확률

In [ ]:
fog = df[["temp","hum","dew","windSpeed"]]

fog_value = fogModel(fog)

In [ ]:
# 안개 모델 돌리고 얻은 확률 column 추가
df['fog'] = fog_value

In [19]:
# 도착, 출발 데이터 분리
df_A = df[df['AOD']=='A']
df_D = df[df['AOD']=='D']

# AOD column삭제 
df_A = df_A.drop(['AOD'],axis = 1)
df_D = df_D.drop(['AOD'],axis = 1)

In [ ]:
# ------------------------------ #

In [ ]:
# df_A 모델 로드 및 실행

In [ ]:
# df_D 모델 로드 및 실행

In [ ]:
# 결과 저장